In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import scipy
# import tushare as ts
import wrds
import talib as ta
import datetime, time

In [5]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

using database

In [6]:
# db = wrds.Connection(wrds_username='kpu2019')

In [12]:
db.list_tables('crsp')

In [30]:
# 这样看格式, 不如直接看数据的来得好
db.describe_table('crsp', 'msf')

Approximately 4509850 rows in crsp.msf.


,name,nullable,type
0,cusip,True,VARCHAR(8)
1,permno,True,DOUBLE PRECISION
2,permco,True,DOUBLE PRECISION
3,issuno,True,DOUBLE PRECISION
4,hexcd,True,DOUBLE PRECISION
5,hsiccd,True,DOUBLE PRECISION
6,date,True,DATE
7,bidlo,True,DOUBLE PRECISION
8,askhi,True,DOUBLE PRECISION
9,prc,True,DOUBLE PRECISION


```python
test = db.get_table('crsp', 'msf', obs=100)
test.head().T

sql_str = 'select * from crsp.msf LIMIT 10'
test = db.raw_sql(sql_str)
test.head().T
```

In [25]:
test.head().T

,0,1,2,3,4
cusip,68391610,68391610,68391610,68391610,68391610
permno,10000,10000,10000,10000,10000
permco,7952,7952,7952,7952,7952
issuno,10396,10396,10396,10396,10396
hexcd,3,3,3,3,3
hsiccd,3990,3990,3990,3990,3990
date,1985-12-31,1986-01-31,1986-02-28,1986-03-31,1986-04-30
bidlo,NaN,-2.5,-3.25,-3.25,-4
askhi,NaN,-4.4375,-4.375,-4.4375,-4.3125
prc,NaN,-4.375,-3.25,-4.4375,-4


In [27]:
for i in test.columns:
    print(i)

cusip
permno
permco
issuno
hexcd
hsiccd
date
bidlo
askhi
prc
vol
ret
bid
ask
shrout
cfacpr
cfacshr
altprc
spread
altprcdt
retx


In [65]:
sql_str = """
select cusip, permno, permco, date, prc, vol, shrout, ret, retx, cfacpr, cfacshr 
from crsp.msf 
where permno in (14593, 90319, 12490, 17778) and date = '2013-01-04'
"""

In [66]:
test = db.raw_sql(sql_str)

In [67]:
test.head()

,cusip,permno,permco,date,prc,vol,shrout,ret,retx,cfacpr,cfacshr


In [14]:
# db = wrds.Connection(wrds_username='kpu2019')
data = db.raw_sql("select a.gvkey, a.datadate, a.tic, a.conm, a.at, a.lt, a.datafmt, \
b.prccm, b.cshoq \
from comp.funda a join comp.secm b \
on a.gvkey = b.gvkey and a.datadate = b.datadate \
where a.tic = 'IBM' and a.datafmt = 'STD' and a.consol = 'C' and a.indfmt = 'INDL'")

In [17]:
db = wrds.Connection(wrds_username='kpu2019')
data = db.raw_sql("""
select a.gvkey, a.datadate, a.tic, a.conm, a.at, a.lt, b.prccm, b.cshoq 
from comp.funda a join comp.secm b on a.gvkey = b.gvkey and a.datadate = b.datadate 
where a.tic = 'IBM' and a.datafmt = 'STD' and a.consol = 'C' and a.indfmt = 'INDL'
""")

Loading library list...
Done


In [19]:
db.close()

In [18]:
data.head()

,gvkey,datadate,tic,conm,at,lt,prccm,cshoq
0,006066,1962-12-31,IBM,INTL BUSINESS MACHINES CORP,2112.301,731.700,389.999567,NaN
1,006066,1963-12-31,IBM,INTL BUSINESS MACHINES CORP,2373.857,782.119,506.999353,NaN
2,006066,1964-12-31,IBM,INTL BUSINESS MACHINES CORP,3309.152,1055.072,409.499496,NaN
3,006066,1965-12-31,IBM,INTL BUSINESS MACHINES CORP,3744.917,1166.771,498.999146,NaN
4,006066,1966-12-31,IBM,INTL BUSINESS MACHINES CORP,4660.777,1338.149,371.499662,NaN


```sql
1. 去重；关键字distinct去重功能  在其他数据库（oracle，mysql）是存在；当然postgresql也有这个功能

[postgres@sdserver40_210 ~]$ psql mydb lottu
psql (9.5.0)
Type "help" for help.

mydb=> select * from trade;
 tradeno | accountid  | fee | game_id 
---------+------------+-----+---------
 1000006 | yyb_100001 |  10 |    2555
 1000011 | yyb_100002 | 100 |    2555
 1001859 | yyb_100001 |  10 |    2555
 1001861 | yyb_100003 |  20 |    2555
 1001854 | yyb_100004 |   6 |    2555
 1001881 | yyb_100002 | 328 |    2555
(6 rows)

mydb=> select distinct accountid from trade;
 accountid  
------------
 yyb_100001
 yyb_100004
 yyb_100002
 yyb_100003
(4 rows)

mydb=> select distinct accountid,game_id from trade;
 accountid  | game_id 
------------+---------
 yyb_100001 |    2555
 yyb_100003 |    2555
 yyb_100004 |    2555
 yyb_100002 |    2555
(4 rows)
2. 跟on一起用； 使用DISTINCT ON实现用窗口函数实现的取第一名的功能
    这个功能oracle，mysql是没有的；当然它们有其他的分析函数可以替换；顶替；例如row_number, fisrt_values等等
mydb=> select distinct on (accountid) accountid,fee from trade;
 accountid  | fee 
------------+-----
 yyb_100001 |  10
 yyb_100002 | 100
 yyb_100003 |  20
 yyb_100004 |   6
(4 rows)

mydb=> select distinct on (game_id) accountid,fee from trade;
 accountid  | fee 
------------+-----
 yyb_100001 |  10
(1 row)

mydb=> select distinct on (game_id) accountid,fee from trade order by game_id, fee desc;
 accountid  | fee 
------------+-----
 yyb_100002 | 328
(1 row)

--例如取每个帐号充值最大的一笔
mydb=> select distinct on (accountid) accountid,fee from trade order by accountid, fee desc;
 accountid  | fee 
------------+-----
 yyb_100001 |  10
 yyb_100002 | 328
 yyb_100003 |  20
 yyb_100004 |   6
(4 rows)
```

In [24]:
db = wrds.Connection(wrds_username='kpu2019')

Loading library list...
Done


In [25]:
data = db.raw_sql("""
create table CUSIP_LINK as 
select distinct a.permno, gvkey, iid, date, prc, vol, ret 
from 
    crsp.msf as a, 
    crsp.msenames 
    (
        where not missing(ncusip) and shrcd in (10 11)
    ) as b, 
    comp.security 
    (
        where not missing(cusip) and excntry='USA'
    ) as c 
where 
    a.permno = b.permno 
    and NAMEDT<=a.date<=NAMEENDT 
    and b.ncusip=substr(c.cusip,1,8)
""")

ProgrammingError: (psycopg2.ProgrammingError) syntax error at or near "where"
LINE 8:         where not missing(ncusip) and shrcd in (10 11)
                ^
 [SQL: "\ncreate table CUSIP_LINK as \nselect distinct a.permno, gvkey, iid, date, prc, vol, ret \nfrom \n    crsp.msf as a, \n    crsp.msenames \n    (\n        where not missing(ncusip) and shrcd in (10 11)\n    ) as b, \n    comp.security \n    (\n        where not missing(cusip) and excntry='USA'\n    ) as c \nwhere \n    a.permno = b.permno \n    and NAMEDT<=a.date<=NAMEENDT \n    and b.ncusip=substr(c.cusip,1,8)\n"] (Background on this error at: http://sqlalche.me/e/f405)

In [ ]:
import 